Temporal regularized matrix factorization (TRMF) for metro OD forecasting. Code is adapted from [https://github.com/xinychen/transdim](https://github.com/xinychen/transdim).

Original paper for TRMF:
- Hsiang-Fu Yu, Nikhil Rao, Inderjit S. Dhillon, 2016. Temporal regularized matrix factorization for high-dimensional time series prediction. 30th Conference on Neural Information Processing Systems (NIPS 2016).

# Define functions

In [5]:
from functions import *
from numpy.linalg import inv as inv
import random
import time


def reset_random_seeds(n=1):
    os.environ['PYTHONHASHSEED'] = str(n)
    np.random.seed(n)
    random.seed(n)


def kr_prod(a, b):
    return np.einsum('ir, jr -> ijr', a, b).reshape(a.shape[0] * b.shape[0], -1)


def TRMF(train_data, init, time_lags, lambda_w, lambda_x, lambda_theta, eta, maxiter, multi_steps=1, display=10):
    start = time.time()
    W = init["W"]
    X = init["X"]
    theta = init["theta"]

    dim1, dim2 = train_data.shape
    binary_mat = np.zeros((dim1, dim2))
    position = np.where((train_data != 0))
    binary_mat[position] = 1

    d = len(time_lags)
    r = theta.shape[1]

    for iter in range(maxiter):
        if (iter + 1) % display == 0:
            print('Time step: {} time {}'.format(iter + 1, time.time() - start))
        var1 = X.T
        var2 = kr_prod(var1, var1)
        var3 = np.matmul(var2, binary_mat.T)
        var4 = np.matmul(var1, train_data.T)
        for i in range(dim1):
            W[i, :] = np.matmul(inv((var3[:, i].reshape([r, r])) + lambda_w * np.eye(r)), var4[:, i])

        var1 = W.T
        var2 = kr_prod(var1, var1)
        var3 = np.matmul(var2, binary_mat)
        var4 = np.matmul(var1, train_data)
        for t in range(dim2):
            Mt = np.zeros((r, r))
            Nt = np.zeros(r)
            if t < max(time_lags):
                Pt = np.zeros((r, r))
                Qt = np.zeros(r)
            else:
                Pt = np.eye(r)
                Qt = np.einsum('ij, ij -> j', theta, X[t - time_lags, :])
            if t < dim2 - np.min(time_lags):
                if t >= np.max(time_lags) and t < dim2 - np.max(time_lags):
                    index = list(range(0, d))
                else:
                    index = list(np.where((t + time_lags >= np.max(time_lags)) & (t + time_lags < dim2)))[0]
                for k in index:
                    theta0 = theta.copy()
                    theta0[k, :] = 0
                    Mt = Mt + np.diag(theta[k, :] ** 2)
                    Nt = Nt + np.multiply(theta[k, :], (X[t + time_lags[k], :]
                                                        - np.einsum('ij, ij -> j', theta0,
                                                                    X[t + time_lags[k] - time_lags, :])))
                X[t, :] = np.matmul(inv(var3[:, t].reshape([r, r])
                                        + lambda_x * Pt + lambda_x * Mt + lambda_x * eta * np.eye(r)),
                                    (var4[:, t] + lambda_x * Qt + lambda_x * Nt))
            elif t >= dim2 - np.min(time_lags):
                X[t, :] = np.matmul(inv(var3[:, t].reshape([r, r]) + lambda_x * Pt
                                        + lambda_x * eta * np.eye(r)), (var4[:, t] + Qt))
        for k in range(d):
            var1 = X[np.max(time_lags) - time_lags[k]: dim2 - time_lags[k], :]
            var2 = inv(np.diag(np.einsum('ij, ij -> j', var1, var1)) + (lambda_theta / lambda_x) * np.eye(r))
            var3 = np.zeros(r)
            for t in range(np.max(time_lags) - time_lags[k], dim2 - time_lags[k]):
                var3 = var3 + np.multiply(X[t, :],
                                          (X[t + time_lags[k], :]
                                           - np.einsum('ij, ij -> j', theta, X[t + time_lags[k] - time_lags, :])
                                           + np.multiply(theta[k, :], X[t, :])))
            theta[k, :] = np.matmul(var2, var3)

    X_new = np.zeros((dim2 + multi_steps, rank))
    X_new[0: dim2, :] = X.copy()
    for step in range(multi_steps):
        X_new[dim2 + step, :] = np.einsum('ij, ij -> j', theta, X_new[dim2 + step - time_lags, :])

    return W, X_new, theta, np.matmul(W, X_new[dim2: dim2 + multi_steps, :].T)


def OnlineTRMF(sparse_vec, init, lambda_x, time_lags):
    W = init["W"]
    X = init["X"]
    theta = init["theta"]
    dim = sparse_vec.shape[0]
    t, rank = X.shape
    position = np.where(sparse_vec != 0)
    binary_vec = np.zeros(dim)
    binary_vec[position] = 1

    xt_tilde = np.einsum('ij, ij -> j', theta, X[t - 1 - time_lags, :])
    var1 = W.T
    var2 = kr_prod(var1, var1)
    var_mu = np.matmul(var1, sparse_vec) + lambda_x * xt_tilde
    inv_var_Lambda = inv(np.matmul(var2, binary_vec).reshape([rank, rank]) + lambda_x * np.eye(rank))
    X[t - 1, :] = np.matmul(inv_var_Lambda, var_mu)
    return X


def st_prediction(train_data, time_lags, lambda_w, lambda_x, lambda_theta, eta,
                  rank, pred_time_steps, maxiter, multi_steps=1, display=100):
    start = time.time()
    start_time = train_data.shape[1] - pred_time_steps
    # dense_mat0 = dense_mat[:, 0: start_time]
    train_data0 = train_data[:, 0: start_time]
    dim1 = train_data0.shape[0]
    dim2 = train_data0.shape[1]
    max_time_lag = max(time_lags)
    results = {step + 1: np.zeros((dim1, pred_time_steps)) for step in range(multi_steps)}

    for t in range(pred_time_steps):
        if t == 0:
            init = {"W": 0.1 * np.random.rand(dim1, rank), "X": 0.1 * np.random.rand(dim2, rank),
                    "theta": 0.1 * np.random.rand(time_lags.shape[0], rank)}
            W, X, theta, mat_f = TRMF(train_data0, init, time_lags,
                                      lambda_w, lambda_x, lambda_theta, eta, maxiter, multi_steps, display=display)
            # Assign forecast to the corresponding step
            for step in range(multi_steps):
                results[step + 1][:, t] = mat_f[:, step]
            X0 = X[dim2 - max_time_lag:dim2 + 1, :].copy()  # Keep recent max_time_lag + one-step forecast
        else:
            sparse_vec = train_data[:, start_time + t - 1]
            if np.where(sparse_vec > 0)[0].shape[0] > rank:
                init = {"W": W, "X": X0, "theta": theta}
                X = OnlineTRMF(sparse_vec, init, lambda_x / dim2, time_lags)
                X0 = np.zeros((max_time_lag + multi_steps, rank))
                X0[0: max_time_lag, :] = X[1:, :]
                for step in range(multi_steps):
                    step_X = np.einsum('ij, ij -> j', theta, X0[max_time_lag + step - time_lags, :])
                    X0[max_time_lag + step, :] = step_X
                    results[step + 1][:, t] = W @ step_X
                X0 = X0[:max_time_lag + 1, :]  # Keep recent max_time_lag + one-step forecast
            else:
                X = X0.copy()
                X0 = np.zeros((max_time_lag + multi_steps, rank))
                X0[0: max_time_lag, :] = X[1:, :]
                for step in range(multi_steps):
                    step_X = np.einsum('ij, ij -> j', theta, X0[max_time_lag + step - time_lags, :])
                    X0[max_time_lag + step, :] = step_X
                    results[step + 1][:, t] = W @ step_X
                X0 = X0[:max_time_lag + 1, :]  # Keep recent max_time_lag + one-step forecast

        if (t + 1) % 40 == 0:
            print('Time step: {}, time {}'.format(t + 1, time.time() - start))
    return results

# Import data

In [6]:
data0 = loadmat('..//data//OD_3m.mat')
data0 = data0['OD']
data0 = remove_weekends(data0, start=5)

train_idx = start_end_idx('2017-07-03', '2017-08-11', weekend=False, night=False)
test_idx = start_end_idx('2017-08-14', '2017-08-25', weekend=False, night=False)
num_s = 159

# Subtract the mean in the training set
data = data0.astype(np.float64)
data_mean = data[:, train_idx].reshape([num_s * num_s, 36, -1], order='F')
data_mean = data_mean.mean(axis=2)
for i in range(65):
    data[:, i * 36:(i + 1) * 36] = data[:, i * 36:(i + 1) * 36] - data_mean

# Parameter tuning
# Tune weights

In [7]:
multi_steps = 1
pred_time_steps = 36 * 10 + (multi_steps - 1)
train_data = data[:, train_idx]
time_lags = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
d = time_lags.shape[0]
maxiter = 300

# Tune weights
eta = 0.03
rank = 40
rmse_list = []
weights = range(1000, 10000, 1000)
reset_random_seeds(1)
for weight in weights:
    lambda_w = weight
    lambda_x = weight
    lambda_theta = weight
    results = st_prediction(train_data, time_lags, lambda_w, lambda_x, lambda_theta,
                            eta, rank, pred_time_steps, maxiter, multi_steps, display=100)
    rmse_list.append(RMSE(train_data[:, -36 * 10:], results[1]))
    print(rmse_list)

best_weight = weights[np.argmin(rmse_list)]
print('best_weight is {}'.format(best_weight))  # was 3000

Time step: 100 time 422.6742262840271
Time step: 200 time 833.2179036140442
Time step: 300 time 1277.8192734718323
Time step: 40, time 1292.9665188789368
Time step: 80, time 1303.8736522197723
Time step: 120, time 1314.7348022460938
Time step: 160, time 1325.6314294338226
Time step: 200, time 1336.5621950626373
Time step: 240, time 1347.5160694122314
Time step: 280, time 1358.4220051765442
Time step: 320, time 1369.421989440918
Time step: 360, time 1380.2815067768097
[3.16668972973313]
Time step: 100 time 444.49519443511963
Time step: 200 time 893.9184141159058
Time step: 300 time 1348.5971925258636
Time step: 40, time 1364.0833823680878
Time step: 80, time 1375.4222617149353
Time step: 120, time 1386.5361912250519
Time step: 160, time 1397.5827927589417
Time step: 200, time 1408.5984964370728
Time step: 240, time 1419.7393562793732
Time step: 280, time 1431.2546632289886
Time step: 320, time 1443.195256471634
Time step: 360, time 1454.570856332779
[3.16668972973313, 3.0305339309035264

## Tune rank

In [8]:
lambda_w = best_weight
lambda_x = best_weight
lambda_theta = best_weight
rmse_list = []
ranks = range(20, 65, 5)
reset_random_seeds(1)
for rank in ranks:
    results = st_prediction(train_data, time_lags, lambda_w, lambda_x, lambda_theta,
                            eta, rank, pred_time_steps, maxiter, multi_steps)
    rmse_list.append(RMSE(train_data[:, -36 * 10:], results[1]))
    print(rmse_list)

best_rank = ranks[np.argmin(rmse_list)]
print("best_rank is {}".format(best_rank))  # was 35

Time step: 100 time 131.31524014472961
Time step: 200 time 265.184494972229
Time step: 300 time 398.04875469207764
Time step: 40, time 401.73542189598083
Time step: 80, time 404.2212748527527
Time step: 120, time 406.5488770008087
Time step: 160, time 408.8296117782593
Time step: 200, time 411.28218030929565
Time step: 240, time 413.61249351501465
Time step: 280, time 415.8463637828827
Time step: 320, time 418.21504759788513
Time step: 360, time 420.62076210975647
[2.8805232149000175]
Time step: 100 time 167.7458872795105
Time step: 200 time 329.2378520965576
Time step: 300 time 484.9009590148926
Time step: 40, time 489.8233585357666
Time step: 80, time 493.1829447746277
Time step: 120, time 496.5103235244751
Time step: 160, time 499.8064503669739
Time step: 200, time 503.27711749076843
Time step: 240, time 506.5420072078705
Time step: 280, time 509.9474878311157
Time step: 320, time 513.2148325443268
Time step: 360, time 516.5734543800354
[2.8805232149000175, 2.8772345682025073]
Time 

# Forcast and save results

In [9]:
best_rank = 45
best_weight = 6000
multi_steps = 3
pred_time_steps = 36 * 10 + (multi_steps - 1)
train_data = data[:, np.concatenate([train_idx, test_idx])]
time_lags = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
rank = best_rank
lambda_w = best_weight
lambda_x = best_weight
lambda_theta = best_weight
eta = 0.03

maxiter = 300
reset_random_seeds(1)
results = st_prediction(train_data, time_lags, lambda_w, lambda_x, lambda_theta,
                        eta, rank, pred_time_steps, maxiter, multi_steps)

mat_hat1 = results[1][:, 2:2 + 360].copy()
mat_hat2 = results[2][:, 1:1 + 360].copy()
mat_hat3 = results[3][:, 0:0 + 360].copy()
for i in range(mat_hat1.shape[1]):
    mat_hat1[:, i] += data_mean[:, i % 36]
    mat_hat2[:, i] += data_mean[:, i % 36]
    mat_hat3[:, i] += data_mean[:, i % 36]

real_OD = data0[:, test_idx]
real_flow = od2flow(real_OD, num_s=num_s)
print('Results of 1-step forecasting:')
predict_flow1 = od2flow(mat_hat1, num_s=num_s)
get_score(real_OD, mat_hat1, real_flow, predict_flow1)

print('Results of 2-step forecasting:')
predict_flow2 = od2flow(mat_hat2, num_s=num_s)
get_score(real_OD, mat_hat2, real_flow, predict_flow2)

print('Results of 3-step forecasting:')
predict_flow3 = od2flow(mat_hat3, num_s=num_s)
get_score(real_OD, mat_hat3, real_flow, predict_flow3)

np.savez_compressed('..//data//Guangzhou_OD_TRMF_step1.npz', data=mat_hat1)
np.savez_compressed('..//data//Guangzhou_OD_TRMF_step2.npz', data=mat_hat2)
np.savez_compressed('..//data//Guangzhou_OD_TRMF_step3.npz', data=mat_hat3)

Time step: 100 time 415.1035301685333
Time step: 200 time 835.7077527046204
Time step: 300 time 1252.8783340454102
Time step: 40, time 1267.6778545379639
Time step: 80, time 1278.3648753166199
Time step: 120, time 1289.1926267147064
Time step: 160, time 1299.974618434906
Time step: 200, time 1310.7396836280823
Time step: 240, time 1321.4582805633545
Time step: 280, time 1332.1303339004517
Time step: 320, time 1342.974592924118
Time step: 360, time 1353.759488582611
Results of 1-step forecasting:
RMSE of OD: 3.1684082205630237
WMAPE of OD: 0.30079949227553543
SMAPE of OD: 1.0784552098939733
MAE of OD: 1.5284633566432184
r2 of OD: 0.9532032169118612


RMSE of flow: 104.5999755859375
WMAPE of flow: 0.06674592941999435
SMAPE of flow: 0.14253036677837372
MAE of flow: 53.926204681396484
r2 of flow: 0.9884410497556592
Results of 2-step forecasting:
RMSE of OD: 3.273830691388082
WMAPE of OD: 0.30897029365243134
SMAPE of OD: 1.083369958543836
MAE of OD: 1.5699819456691453
r2 of OD: 0.9500372690